# Total Production Units for Self-Consumption

### Master in Data Science and Engineering - FEUP

#### Group 4
202107955 - Beatriz Iara Nunes Silva
\
202206252 - Inês Clotilde da Costa Neves
\
202502527 - Kirill Savin
\
202502528 - Mariana Rocha Cristino
\
202202895 - Patrícia Crespo da Silva

<div id="top"></div>

# Table of Content

<ol>
    <li><h3><a href="#introduction">Introduction</a></h3></li>
    <li><h3><a href="#research">Research Questions</a></h3></li>
    <li><h3><a href="#data">Data</a></h3></li>
    <li><h3><a href="#inference">Inferences</a></h3></li>
    <li><h3><a href="#results">Results and Discussion</a></h3></li>
    <li><h3><a href="#conclusion">Conclusion</a></h3></li>    

</ol>

# Introduction

<div id="research"></div>
<strong><a href="#top">Back to top</a></strong>

# Research Questions

**General Research Question:**
* RQ: Compare how seasonal (winter vs summer), regional, and technical factors shape self-consumption energy production patterns in Portugal between 2023 and 2024.

**Specifics Research Questions:**
* RQ1: Compare the average installed capacity per UPAC across different power levels and districts in 2023 and 2024. - Kirill
* RQ2: Compare the evolution of installed capacity between 2023 and 2024 across residential and industrial UPACs to assess differences in growth patterns. - Iara and Mariana
* RQ3: Compare the total installed capacity for self-consumption across different power scales (installed capacity ranges) and seasons (winter vs. summer) in selected Portuguese districts during 2023 and 2024. - Inês and Patrícia

Districts for SRQ: Aveiro, Évora, Vila Real and Faro

<div id="research"></div>
<strong><a href="#top">Back to top</a></strong>

# Data

## Libraries

In [ ]:
import geopandas as gpd
import json
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from matplotlib import patheffects
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
import statsmodels

## Prepare Data

Reading the dataset:

In [ ]:
df_origin = pd.read_csv('../Data/UPAC_Total_Production.csv', sep=';', decimal='.')
df_origin.head(10)

Rename the last two columns to English

In [ ]:
df_origin = df_origin.rename(columns={
    "relacao_instalacoes_por_cpe": "installations_per_cpe_ratio",
    "relacao_potencia_por_cpe": "power_per_cpe_ratio"
})
print(df_origin.columns)

Dataset info

In [ ]:
print("\nDataset info:")
print(df_origin.info())

Non-Numeric Columns

In [ ]:
non_numeric = df_origin.select_dtypes(exclude=['number'])


for col in non_numeric.columns:
    unique_vals = df_origin[col].unique().tolist()
    print(f"Column: {col} — {len(unique_vals)} unique values")
    print(unique_vals)
    print("-" * 40)

Translation Dictionary Technology Type

In [ ]:
tech_type_translation = {
    'Solar': 'Solar',
    'Não Atribuído': 'Not Assigned',
    'Eólica': 'Wind',
    'Biogás': 'Biogas',
    'Cogeração não renovável': 'Non-renewable Cogeneration',
    'Hídrica': 'Hydro',
    'Biomassa': 'Biomass',
    'Fotovoltaica': 'Photovoltaic'
}

df_origin['Technology Type'] = df_origin['Technology Type'].map(tech_type_translation).fillna(df_origin['Technology Type'])
print(df_origin['Technology Type'].unique())


### Data Selection

Removing unnecessary columns

In [ ]:
cols_to_keep = [
    "Quarter",
    "District",
    "Municipality",
    "Parish",
    "Technology Type",
    "Voltage level",
    "Installed power range (kW)",
    "Number of installations",
    "Total installed power (kW)",
    "CPEs (#)",
    "installations_per_cpe_ratio",
    "power_per_cpe_ratio"
]

df_filtered = df_origin[cols_to_keep]
df_filtered.head(10)

Selecting years of interest

In [ ]:
df_filtered = df_filtered[df_filtered['Quarter'].str.startswith(('2023', '2024'))].copy()
print(df_filtered['Quarter'].value_counts())

### Data Inspection and Cleaning

Checking for missing values

In [ ]:
missing_df = pd.DataFrame({
    'Missing Values': df_filtered.isnull().sum(),
    'Percentage': (df_filtered.isnull().sum() / len(df_filtered)) * 100
})
print("\nMissing Values summary:")
display(missing_df[missing_df['Missing Values'] > 0])

In [ ]:
print("Before dropping NA:", df_filtered.shape)
df_filtered = df_filtered.dropna()
print("After dropping NA:", df_filtered.shape)

Check for duplicate rows

In [ ]:
duplicates = df_filtered[df_filtered.duplicated()]
print(duplicates)
print("Number of duplicates:", df_filtered.duplicated().sum())

In [ ]:
print("Before dropping duplicates:", df_filtered.shape)
df_filtered = df_filtered.drop_duplicates()
print("After dropping duplicates:", df_filtered.shape)

### Derived Metrics

**Percentage of Installations by Voltage Level per District**

According to E-redes ([Manual de Ligações à Rede](https://provedordocliente.e-redes.pt/Files/PDF/Manual-de-Ligacoes-a-Rede.pdf)):

*High Voltage (AT)*
- Companies with capacities >10 MVA, supplied at 60 kV.
- Clear proxy for heavy industry or large commercial facilities.
- Districts with a higher proportion of AT installations → more industrialized.

*Medium Voltage (MT)*
- Companies with capacities >200 kVA, voltages of 10–30 kV.
- Also indicates industrial activity or large commercial companies.
- Complements AT; districts with a higher proportion of MT → more industrialized areas, but less intense than AT.

*Low Voltage Normal (BTN) and Low Voltage Special (BTE)*
- **BTN** → residences, small shops/offices.
- **BTE** → small/medium companies (>41.4 kVA).
- Districts dominated by BTN → mainly residential areas.
- BTE is mixed, can indicate areas with small industries or commerce, but less significant than MT/AT.


In [ ]:
# Make a copy
df = df_filtered.copy()

# Group by Quarter, District, and Voltage level, summing the Number of installations
grouped = (
    df.groupby(['Quarter', 'District', 'Voltage level'], as_index=False)['Number of installations'].sum()
)

# Pivot table to have Voltage levels as columns
pivot = grouped.pivot_table(
    index=['Quarter', 'District'],
    columns='Voltage level',
    values='Number of installations',
    fill_value=0
).reset_index()

# Ensure all expected voltage columns exist
for col in ['AT', 'MT', 'BTN', 'BTE']:
    if col not in pivot.columns:
        pivot[col] = 0

# Calculate total installations per row
pivot['Total'] = pivot[['AT','MT','BTN','BTE']].sum(axis=1)

# Calculate percentage per voltage level
pivot["District_High_Voltage_AT(%)"] = pivot['AT'] / pivot['Total'] * 100
pivot["District_Medium_Voltage_MT(%)"] = pivot['MT'] / pivot['Total'] * 100
pivot["District_Low_Voltage_BTN(%)"] = pivot['BTN'] / pivot['Total'] * 100
pivot["District_Low_Voltage_BTE(%)"] = pivot['BTE'] / pivot['Total'] * 100

# Select only the relevant columns
df_result = pivot[['Quarter', 'District',
                   'District_High_Voltage_AT(%)',
                   'District_Medium_Voltage_MT(%)',
                   'District_Low_Voltage_BTN(%)',
                   'District_Low_Voltage_BTE(%)']]

df_result


In [ ]:
df_final = df_filtered.merge(df_result, on=['Quarter', 'District'], how='left')
df_final.head(10)

**Map Quarters to Seasons**

In [ ]:
# Function to convert Quarter to Season
def quarter_to_season(quarter):
    if quarter.endswith('T1') or quarter.endswith('T2'):
        return 'Winter'
    elif quarter.endswith('T3') or quarter.endswith('T4'):
        return 'Summer'
    else:
        return 'Unknown'

# Apply the function to create a new Season column
df_final['Season'] = df_final['Quarter'].apply(quarter_to_season)

# Display first 10 rows
df_final.head(10)


Check final data types

In [ ]:
df_final.dtypes

**Summary of numeric variables**

In [ ]:
df = df_final.copy()
df.describe().T

**Number of installations**

Average: 17.47 → on average, each record has ~18 installations.

Standard deviation: 58.11 → high, indicating great variability between municipalities/records.

Distribution: median = 2, 25% = 1, 75% = 8 → asymmetric: most records have few installations, but some have extremely high values (max = 2118), possibly outliers or large production centers.

**Total Installed Power UPAC (kW)**

Average: 122.10 kW → relatively low considering that there are large outliers.

Standard deviation: 383.69 kW → very high, again showing strong variability.

Quartiles: 25% = 14.04 kW, median = 30.00 kW, 75% = 82.79 kW

Maximum: 19,600 kW → indicates the existence of some very large installations.

Conclusion: most units are small, but there are large installations that greatly increase the average.

# Data Exploration

Added another dataset to enable visualization of information on maps.

In [ ]:
map = gpd.read_file("../Data/Portugal_Map.gpkg", layer='cont_distritos')

print(map.columns)
print(map.head())

map = map[['distrito', 'geometry']]

In [ ]:
# Select only numeric columns
num_cols = df.select_dtypes(include=['number']).columns.tolist()

# Correlation matrix
corr_matrix = df[num_cols].corr(method='pearson')

# Visualization with heatmap
plt.figure(figsize=(8,6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Pearson Correlation between Numeric Variables')
plt.show()

**Distribution of Total Installed Power (Log-transformed)**

To better visualize the highly skewed distribution of *'Total Installed Power (kW)'*, we applied a *log10 transformation*:

In [ ]:
# Transform Total Installed Power using log10
df['LogPower'] = np.log10(df['Total installed power (kW)'] + 1e-3)  # add small epsilon to avoid log(0)

The log transformation compresses the extreme high values, bringing them closer to the rest of the data, while preserving the relative differences. This makes the histogram and boxplot more interpretable, especially for a dataset with very skewed values and large outliers.

In [ ]:
sns.set(style='whitegrid')

# Determine min/max for axis scaling (optional, for consistency)
x_min = df['LogPower'].min()
x_max = df['LogPower'].max()

# Create figure
fig = plt.figure(figsize=(10, 6))
gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.30)

# --- HISTOGRAM ---
ax0 = plt.subplot(gs[0])
sns.histplot(
    data=df,
    x='LogPower',
    bins=100,
    color='#2159B4',
    kde=True,
    ax=ax0
)
ax0.set_title('Distribution of Total Installed Power (UPAC)', fontsize=14, pad=15)
ax0.set_xlabel('')
ax0.set_ylabel('Frequency')

# Convert ticks back to original scale for readability
ticks = ax0.get_xticks()
ax0.set_xticklabels([f"{10**tick:.0f}" for tick in ticks])
ax0.grid(alpha=0.3)

# --- BOXPLOT ---
ax1 = plt.subplot(gs[1])
sns.boxplot(
    data=df,
    x='LogPower',
    color='#2159B4',
    ax=ax1
)
ax1.set_xlabel('Total Installed Power (kW)')
ax1.set_yticks([])

# Match x-axis to histogram and convert ticks back to original scale
ax1.set_xlim(x_min, x_max)
ticks = ax1.get_xticks()
ax1.set_xticklabels([f"{10**tick:.0f}" for tick in ticks])
ax1.grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

This transformation reduces the effect of extreme outliers and makes the distribution more symmetric, allowing us to clearly see patterns in the majority of data points.

**Distribution of Total Installed Power by Voltage Level and Season**

Since the total installed power depends on the voltage level, we performed a detailed analysis by Voltage Level (AT, MT, BTN, BTE) and Season (Winter and Summer).


In [ ]:
# Descriptive statistics by Voltage Level and Season (original scale)
desc_stats = df.groupby(['Season', 'Voltage level'])['Total installed power (kW)'].describe()
print(desc_stats)

Key observations:

- **AT** (High Voltage) shows the highest mean power (~1,500–1,600 kW) and extreme maximum values (~19,600 kW), reflecting large installations.

- **MT** (Medium Voltage) also has high mean values (~360–380 kW), but lower than AT.

- **BTN** and **BTE** (Low Voltage) have much lower mean installed power (~55–64 kW).

**Crucially**, differences between Winter and Summer seem minimal for most voltage levels, except for slight variations in AT and MT due to seasonal installations or operational patterns.


We visualized the distributions using log-transformed histograms and boxplots, which confirm these patterns:

- Heavy skew and long tails for AT and MT.

- Concentrated low values for BTN and BTE.

- Seasonal shifts seem minor except in high-voltage levels.

In [ ]:
sns.set(style='whitegrid')

# Ensure columns exist
df['Year'] = df['Quarter'].str[:4]

# Color palette for voltage levels
palette = {'AT': '#1f77b4', 'MT': '#ff7f0e', 'BTN': '#2ca02c', 'BTE': '#d62728'}

# Order of categories
voltage_levels = ['AT', 'MT', 'BTN', 'BTE']
seasons = ['Winter', 'Summer']

# Global scale (log)
x_min = df['LogPower'].min()
x_max = df['LogPower'].max()

# Figure setup: 4 columns (voltage) × 2 rows (seasons)
fig = plt.figure(figsize=(4 * len(voltage_levels), 10))
outer_gs = fig.add_gridspec(2, len(voltage_levels), hspace=0.4, wspace=0.25)

# Loop through each Season (row) and Voltage level (column)
for row, season in enumerate(seasons):
    subset_season = df[df['Season'] == season]

    for col, voltage in enumerate(voltage_levels):
        subset = subset_season[subset_season['Voltage level'] == voltage]

        # Inner gridspec for histogram + boxplot (stacked vertically)
        inner_gs = outer_gs[row, col].subgridspec(2, 1, height_ratios=[4, 1], hspace=0.25)

        # --- HISTOGRAM ---
        ax_hist = fig.add_subplot(inner_gs[0])
        sns.histplot(
            data=subset,
            x='LogPower',
            bins=50,
            color=palette[voltage],
            kde=False,
            ax=ax_hist
        )
        if row == 0:
            ax_hist.set_title(voltage, fontsize=12)
        if col == 0:
            ax_hist.set_ylabel(f'{season}\nCount')
        else:
            ax_hist.set_ylabel('')
        ax_hist.set_xlabel('')
        ax_hist.set_xlim(x_min, x_max)
        ticks = ax_hist.get_xticks()
        ax_hist.set_xticks(ticks)
        ax_hist.set_xticklabels([f"{10**tick:.0f}" for tick in ticks])
        ax_hist.grid(alpha=0.3)

        # --- BOX PLOT ---
        ax_box = fig.add_subplot(inner_gs[1])
        sns.boxplot(
            data=subset,
            x='LogPower',
            color=palette[voltage],
            ax=ax_box
        )
        ax_box.set_xlabel('Total Installed Power (kW)')
        ax_box.set_yticks([])
        ax_box.set_xlim(x_min, x_max)
        ticks = ax_box.get_xticks()
        ax_box.set_xticks(ticks)
        ax_box.set_xticklabels([f"{10**tick:.0f}" for tick in ticks])
        ax_box.grid(alpha=0.3, axis='x')

# Global title
plt.suptitle('Distribution of Total Installed Power by Voltage Level and Season (log scale)', fontsize=16, y=0.98)
plt.tight_layout()
plt.show()


**Lineplot by Voltage Level – Total Installed Power per Voltage Level per Quarter**

In [ ]:
df_time = df.groupby(['Quarter', 'Voltage level'], as_index=False)['Total installed power (kW)'].sum()
df_voltage = df_time.groupby(['Quarter','Voltage level'])['Total installed power (kW)'].sum().reset_index()

fig9 = px.line(
    df_voltage,
    x='Quarter',
    y='Total installed power (kW)',
    color='Voltage level',
    facet_col='Voltage level',
    title='Total Installed Power per Voltage Level per Quarter',
    markers=True
)

fig9.update_layout(template='plotly_white', title_x=0.5, showlegend=False)
fig9.show()


**Violin Plot of Total Installed Power by Voltage Level**

In [ ]:
voltage_levels = df['Voltage level'].unique()

fig2 = make_subplots(
    rows=1, cols=len(voltage_levels),
    shared_yaxes=False,
    subplot_titles=voltage_levels
)

for i, voltage in enumerate(voltage_levels, start=1):
    df_voltage = df[df['Voltage level'] == voltage]
    fig2.add_trace(
        go.Box(
            y=df_voltage['Total installed power (kW)'],
            name=voltage,
            boxpoints='all',
            marker_color=px.colors.qualitative.Vivid[i % len(px.colors.qualitative.Vivid)],
            line=dict(width=0),
            fillcolor='rgba(0,0,0,0)'
        ),
        row=1, col=i
    )

fig2.update_layout(
    template='plotly_white',
    title='Total Installed Power by Voltage Level',
    title_x=0.5,
    showlegend=False,
    height=500,
    width=300*len(voltage_levels)
)
fig2.show()

**Total Installed Power per District Over Time**

In [ ]:
agg_df = df.groupby(['Quarter', 'District', 'Technology Type', 'Voltage level',	'Installed power range (kW)'], as_index=False).agg({
    'Number of installations': 'sum',
    'Total installed power (kW)': 'sum'
})

agg_district_df = agg_df.groupby(['Quarter', 'District'], as_index=False).agg({
    'Number of installations': 'sum',
    'Total installed power (kW)': 'sum'
})

y_max = agg_district_df['Total installed power (kW)'].max() * 1.1

fig = px.bar(
    agg_district_df,
    x='District',
    y='Total installed power (kW)',
    color='District',
    animation_frame='Quarter',
    title='Total Installed Power per District Over Time'
)

fig.update_yaxes(range=[0, y_max])
fig.show()


**Percentage Change of Total Installed Power per District**

In [ ]:
map_gdf = map.to_crs(epsg=4326)

quarters = sorted(df['Quarter'].unique())

all_changes = []
for i in range(len(quarters) - 1):
    start_q, end_q = quarters[i], quarters[i + 1]
    df_start = df[df['Quarter'] == start_q].groupby('District')['Total installed power (kW)'].sum()
    df_end   = df[df['Quarter'] == end_q].groupby('District')['Total installed power (kW)'].sum()
    df_growth = ((df_end - df_start) / df_start * 100).reset_index()
    df_growth.columns = ['District', 'Perc_Change']
    df_growth['Quarter Change'] = f"{start_q} → {end_q}"
    all_changes.append(df_growth)

growth_df = pd.concat(all_changes, ignore_index=True)

map_merged = map_gdf.merge(growth_df, left_on='distrito', right_on='District', how='left')

geojson = json.loads(map_gdf.to_json())

fig = px.choropleth(
    map_merged,
    geojson=geojson,
    locations='District',
    featureidkey='properties.distrito',
    color='Perc_Change',
    animation_frame='Quarter Change',
    hover_name='District',
    hover_data={'Perc_Change': ':.2f'},
    color_continuous_scale='Viridis',
    title='Quarterly Percentage Change of Total Installed Power per District'
)

fig.update_geos(
    fitbounds="locations",
    visible=False,
    projection_type="mercator"
)
fig.update_layout(
    width=950,
    height=750,
    margin=dict(l=0, r=0, t=70, b=0),
    coloraxis_colorbar=dict(title='Percentage Change (%)')
)

fig.show()


**Distribution of Number of Installtions by Voltage Level**

To better understand how the *'Number of Installations'* varies across different Voltage Levels, we first computed descriptive statistics:



In [ ]:
# Descriptive statistics for 'Number of Installations' by Voltage Level
desc_stats_installations = df.groupby('Voltage level')['Number of installations'].describe()
print(desc_stats_installations)


- **AT** (High Voltage): Very few installations, mostly 1 per unit, with a maximum of 2.

- **MT** (Medium Voltage): Low number of installations, median = 1, maximum = 58.

- **BTN** (Low Voltage): Largest number of installations, highly skewed distribution with some extreme outliers (max = 2118).

- **BTE** (Low Voltage): Mostly small numbers of installations, median = 1, maximum = 19.

These statistics highlight a highly skewed distribution, especially for BTN, where a few units account for a disproportionately large number of installations.

In [ ]:
sns.set(style='whitegrid')

# Log-transform Number of Installations (add small epsilon to avoid log(0))
df['LogInstallations'] = np.log10(df['Number of installations'] + 1e-3)

# Color palette for Voltage levels
palette = {'AT': '#1f77b4', 'MT': '#ff7f0e', 'BTN': '#2ca02c', 'BTE': '#d62728'}
voltage_levels = ['AT', 'MT', 'BTN', 'BTE']
n_cols = len(voltage_levels)

# Determine global min/max for consistent axis scaling
x_min = df['LogInstallations'].min()
x_max = df['LogInstallations'].max()

# Create figure with gridspec
fig = plt.figure(figsize=(4 * n_cols, 6))
gs = fig.add_gridspec(2, n_cols, height_ratios=[4, 1], hspace=0.3, wspace=0.25)

for i, voltage in enumerate(voltage_levels):
    subset = df[df['Voltage level'] == voltage]

    # --- HISTOGRAM ---
    ax_hist = fig.add_subplot(gs[0, i])
    sns.histplot(
        data=subset,
        x='LogInstallations',
        bins=50,
        color=palette[voltage],
        kde=False,
        ax=ax_hist
    )
    ax_hist.set_title(voltage, fontsize=12)
    ax_hist.set_xlabel('')
    ax_hist.set_ylabel('Count' if i == 0 else '')
    ax_hist.set_xlim(x_min, x_max)

    # Convert log-ticks back to original scale
    ticks = ax_hist.get_xticks()
    ax_hist.set_xticks(ticks)
    ax_hist.set_xticklabels([f"{10**tick:.0f}" for tick in ticks])
    ax_hist.grid(alpha=0.3, which='both')

    # --- BOX PLOT ---
    ax_box = fig.add_subplot(gs[1, i])
    sns.boxplot(
        data=subset,
        x='LogInstallations',
        color=palette[voltage],
        ax=ax_box
    )
    ax_box.set_xlabel('Number of Installations')
    ax_box.set_yticks([])
    ax_box.set_xlim(x_min, x_max)

    ticks = ax_box.get_xticks()
    ax_box.set_xticks(ticks)
    ax_box.set_xticklabels([f"{10**tick:.0f}" for tick in ticks])
    ax_box.grid(alpha=0.3, axis='x')

plt.suptitle('Distribution of Number of Installations by Voltage Level (log scale)', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()


From the plots:

- BTN clearly shows the widest range of installations.

- AT, MT, and BTE have distributions concentrated at very low values.



**District-level Installations vs Power Over Time**

In [ ]:
agg_df = df.groupby(['Quarter', 'District', 'Technology Type', 'Voltage level',	'Installed power range (kW)'], as_index=False).agg({
    'Number of installations': 'sum',
    'Total installed power (kW)': 'sum'
})

agg_district_df = agg_df.groupby(['Quarter', 'District'], as_index=False).agg({
    'Number of installations': 'sum',
    'Total installed power (kW)': 'sum'
})

# Get global axis limits
x_min = agg_district_df['Number of installations'].min() * 0.9
x_max = agg_district_df['Number of installations'].max() * 1.1
y_min = agg_district_df['Total installed power (kW)'].min() * 0.9
y_max = agg_district_df['Total installed power (kW)'].max() * 1.1

fig = px.scatter(
    agg_district_df,
    x='Number of installations',
    y='Total installed power (kW)',
    color='District',
    size='Total installed power (kW)',
    hover_name='District',
    animation_frame='Quarter',
    title='District-level Installations vs Power Over Time'
)

# Fix axes to always show all bubbles
fig.update_xaxes(range=[x_min, x_max])
fig.update_yaxes(range=[y_min, y_max])

fig.show()


**Average Power per Installation vs Number of Installations Over Time**

In [ ]:
# Prepare data
agg_df = df.groupby(['Quarter', 'District'], as_index=False).agg({
    'Number of installations': 'sum',
    'Total installed power (kW)': 'sum'
})
agg_df['Average power per installation'] = (
    agg_df['Total installed power (kW)'] / agg_df['Number of installations']
)

x_min = agg_df['Number of installations'].min() * 0.9
x_max = agg_df['Number of installations'].max() * 1.1
y_min = agg_df['Average power per installation'].min() * 0.9
y_max = agg_df['Average power per installation'].max() * 1.1

fig_scatter = px.scatter(
    agg_df,
    x='Number of installations',
    y='Average power per installation',
    color='District',
    size='Number of installations',
    hover_name='District',
    animation_frame='Quarter',
    title='Average Power per Installation vs Number of Installations',
    size_max=40
)
fig_scatter.update_xaxes(range=[x_min, x_max])
fig_scatter.update_yaxes(range=[y_min, y_max])
fig_scatter.update_layout(
    height=500,
    margin=dict(l=50, r=50, t=80, b=50)
)

In [ ]:
# Prepare heatmap
average_production = df.groupby(['Quarter', 'District'])['Number of installations'].mean().reset_index()
pivot = average_production.pivot(index='District', columns='Quarter', values='Number of installations')

fig_heatmap = go.Figure(data=go.Heatmap(
    z=pivot.values,
    x=pivot.columns,
    y=pivot.index,
    colorscale='YlGnBu',
    text=pivot.values,
    texttemplate="%{text:.2f}",
    colorbar=dict(title='Average Number of Installations'),
    xgap=1,
    ygap=1
))
fig_heatmap.update_layout(
    title='Average Number of Installations per District Over Time',
    xaxis_title='Quarter',
    yaxis_title='District',
    yaxis=dict(autorange='reversed'),
    height=500,
    margin=dict(l=80, r=40, t=60, b=50)
)

### Relationship Between Total Installed Power and Number of Installations by Voltage Level

We start by exploring the overall relationship between Total Installed Power and Number of Installations.

When we plot Log(Total Installed Power) against Log(Number of Installations) without distinguishing voltage levels, the scatter plot shows a general positive trend but also suggests the presence of distinct clusters. These clusters appear as roughly vertical “stripes” along the x-axis, indicating that different Voltage Levels may be driving separate groups of points.

This observation motivates a second visualization in which points are colored according to Voltage Level, allowing us to confirm whether the clusters correspond to different voltage categories and to better understand the relationship within each group.

In [ ]:
sns.set(style='whitegrid')
plt.figure(figsize=(8,6))

palette = {'AT': '#1f77b4', 'MT': '#ff7f0e', 'BTN': '#2ca02c', 'BTE': '#d62728'}

# Scatter plot
scatter = sns.scatterplot(
    x='LogPower',
    y='LogInstallations',
    hue='Voltage level',
    data=df,
    palette=palette,
    alpha=0.6
)

# Regression lines and coefficients
for voltage, color in palette.items():
    subset = df[df['Voltage level'] == voltage]
    X = sm.add_constant(subset['LogPower'])
    y = subset['LogInstallations']
    model = sm.OLS(y, X).fit()
    a, b = model.params
    print(f"{voltage} regression: Intercept = {a:.3f}, Slope = {b:.3f}")

    # Plot regression line (without label)
    x_vals = subset['LogPower']
    y_vals = a + b * x_vals
    plt.plot(x_vals, y_vals, color=color)

plt.xlabel('Log(Total Installed Power)')
plt.ylabel('Log(Number of Installations)')
plt.title('Scatter Plot with Regression Lines by Voltage Level')

# Move legend outside
plt.legend(title='Voltage Level', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

This visualization confirms that the apparent clusters in the overall plot are indeed due to different Voltage Levels. 

The regression coefficients highlight distinct patterns:

- AT (High Voltage) installations show almost no increase in the number of installations with increasing power, indicating very few, large installations; 

- MT (Medium Voltage) and BTE (Low Voltage) exhibit moderate positive relationships; 

- BTN (Low Voltage) shows a strong positive relationship, meaning that as total installed power increases, the number of installations grows significantly. 

These results confirm that Voltage Level might be a key factor influencing both the total installed power and the number of installations, and analyzing the data without accounting for voltage would mask these differences.

### Number of Installations by District and Season (Top 5 Districts)

In [ ]:
sns.set_theme(style='whitegrid')
# Function to prepare stacked bar data by Season
def prepare_data_season(df, year):
    df_year = df[df['Quarter'].str.startswith(str(year))]
    # Top 5 districts with most installations
    top_districts = df_year.groupby('District')['Number of installations'].sum().sort_values(ascending=False).head(5).index

    # Prepare data for stacked bar
    stack_data = df_year[df_year['District'].isin(top_districts)]
    stack_data = stack_data.groupby(['District', 'Season'])['Number of installations'].sum().unstack(fill_value=0)

    seasons_order = ['Summer', 'Winter']  # English translation
    stack_data = stack_data.reindex(columns=seasons_order, fill_value=0)

    return stack_data, seasons_order

colors = ["#f15e47", "#5e83f4"]

stack_2023, seasons_2023 = prepare_data_season(df_final, 2023)
stack_2024, seasons_2024 = prepare_data_season(df_final, 2024)

fig, axes = plt.subplots(ncols=2, figsize=(16, 6), sharey=True)

for ax, stack_data, year, seasons_order in zip(axes, [stack_2023, stack_2024], [2023, 2024], [seasons_2023, seasons_2024]):
    left = pd.Series([0]*len(stack_data), index=stack_data.index)
    for i, season in enumerate(seasons_order):
        ax.barh(stack_data.index, stack_data[season], left=left, color=colors[i], label=season, height=0.5)

        for idx, value in enumerate(stack_data[season]):
            if value > 0:
                ax.text(left[idx] + value/2, idx, str(value), va='center', ha='center', color='white', fontsize=9)
        left += stack_data[season]

    ax.set_xlabel('Number of Installations')
    ax.set_title(f'Number of Installations by District (Top 5) - {year}', fontsize=14)
    ax.grid(alpha=0.3, axis='x')

axes[1].legend(title='Season')
plt.tight_layout()
plt.show()


**Number of Installations per District Over Time**

In [ ]:
agg_df = df.groupby(['Quarter', 'District', 'Technology Type', 'Voltage level',	'Installed power range (kW)'], as_index=False).agg({
    'Number of installations': 'sum',
    'Total installed power (kW)': 'sum'
})

agg_district_df = agg_df.groupby(['Quarter', 'District'], as_index=False).agg({
    'Number of installations': 'sum',
    'Total installed power (kW)': 'sum'
})

agg_district_df = agg_district_df.sort_values(['Quarter', 'Number of installations'], ascending=[True, False])

y_max = agg_district_df['Number of installations'].max() * 1.1

districts = agg_district_df['District'].unique()

# Generate viridis colors for each district
cmap = plt.get_cmap('viridis')
viridis_colors = [f"rgb{tuple(int(255*x) for x in cmap(i / (len(districts)-1))[:3])}" for i in range(len(districts))]

color_discrete_map = {district: viridis_colors[i] for i, district in enumerate(districts)}

fig = px.bar(
    agg_district_df,
    x='District',
    y='Number of installations',
    color='District',
    animation_frame='Quarter',
    title='Number of Installations per District Over Time',
    color_discrete_map=color_discrete_map
)

fig.update_yaxes(range=[0, y_max])
fig.show()


### Proportion of Voltage Level within Each Installed Power Range

In [ ]:
cross_tab = pd.crosstab(df['Installed power range (kW)'], df['Voltage level'])

prop_table = (cross_tab.T / cross_tab.T.sum()).T

ax = prop_table.plot(
    kind='bar',
    stacked=True,
    figsize=(10,6),
    colormap='tab10'
)

plt.title('Proportion of Voltage Level within each Installed Power Range', fontsize=14)
plt.ylabel('Proportion')
plt.xlabel('Installed Power Range (kW)')
plt.grid(alpha=0.3, axis='y')

plt.legend(
    title='Voltage Level',
    bbox_to_anchor=(1.05, 0.5),
    loc='center left',
    borderaxespad=0.
)

plt.tight_layout()
plt.show()


### Industrialization Analysis by District

In order to assess the level of industrialization across Portuguese districts, we combined geographical data with electricity installation data, using the proportion of installations by voltage level as a proxy for industrial activity.

High-voltage (AT) and medium-voltage (MT) installations typically correspond to larger companies and industrial facilities, while low-voltage installations (BTE/BTN) mostly represent residential or small commercial areas. By weighting these percentages, we can estimate a district-level industrialization index.

We used the [GADM shapefile for Portugal](https://forest-gis.com/shapefiles-de-portugal/?srsltid=AfmBOorEhHkqSdD7e8js7ueV5iM9g8aRcjPQzEnLZwh5vPB5gVdl6Yl6) and extracted district-level polygons:


To explore the spatial distribution of installations across Portugal, we created choropleth maps showing the average percentage of installations per voltage level in each district. We visualized four categories: High Voltage (AT), Medium Voltage (MT), Low Voltage Normal (BTN), and Low Voltage Special (BTE).

These maps reveal the predominant type of installations in each district:

- Districts with higher AT percentages typically host heavy industrial or large commercial facilities.

- Districts dominated by MT installations indicate medium-to-large industrial activity.

- High percentages of BTN installations correspond mainly to residential areas and small businesses.

- BTE shows the presence of small to medium companies that exceed typical residential consumption.

Overall, these maps allow a quick visual assessment of which districts are more industrialized versus predominantly residential, highlighting regional differences in electricity consumption profiles.

In [ ]:
percent_vars = [
    'District_High_Voltage_AT(%)',
    'District_Medium_Voltage_MT(%)',
    'District_Low_Voltage_BTN(%)',
    'District_Low_Voltage_BTE(%)'
]

df_district_pct = df.groupby('District')[percent_vars].mean()

gdf_plot = map.merge(df_district_pct, left_on='distrito', right_on='District', how='left')
fig, axes = plt.subplots(1, 4, figsize=(24, 8))

cmap = 'OrRd'

for ax, var in zip(axes, percent_vars):
    gdf_plot.plot(
        column=var,
        cmap=cmap,
        linewidth=0.8,
        ax=ax,
        edgecolor='0.8',
        legend=True
    )
    ax.set_title(var.replace('District_', '').replace('_', ' '), fontsize=14)
    ax.axis('off')

plt.tight_layout()
plt.show()

To better highlight the more industrialized districts, we created a weighted Industrialization Index combining the percentages of AT, MT, and BTE installations (weights: AT=3, MT=2, BTE=1). This index emphasizes areas with a higher concentration of industrial or large commercial activity. The corresponding choropleth map allows a clear comparison across districts, making it easier to identify the top and bottom regions in terms of industrial activity.

In [ ]:
sns.set(style='whitegrid')
percent_vars = [
    'District_High_Voltage_AT(%)',
    'District_Medium_Voltage_MT(%)',
    'District_Low_Voltage_BTE(%)'
]


df_district = df.groupby('District')[percent_vars].mean()

# weights: AT=3, MT=2, BTE=1
df_district['Industrialization_Index'] = (
    3 * df_district['District_High_Voltage_AT(%)'] +
    2 * df_district['District_Medium_Voltage_MT(%)'] +
    1 * df_district['District_Low_Voltage_BTE(%)']
)

df_district['Industrialization_Index_norm'] = 100 * df_district['Industrialization_Index'] / df_district['Industrialization_Index'].max()

gdf_plot = map.merge(df_district, left_on='distrito', right_on='District', how='left')
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf_plot.plot(
    column='Industrialization_Index_norm',
    cmap='OrRd',
    linewidth=0.8,
    ax=ax,
    edgecolor='0.8',
    legend=True,
    legend_kwds={'label': "Industrialization Index (%)", 'shrink': 0.6}
)

ax.set_title('Industrialization by District (Weighted Voltage Levels)', fontsize=16)
ax.axis('off')

plt.show()


### Evolution of Installed Capacity by UPAC Type

We then compared the total installed power across the Top 5 most industrialized and Top 5 most residential districts, segmented by UPAC type (Industrial vs Residential) for 2023 and 2024.

In [ ]:
# Variáveis de interesse
residential_vars = [
    'District_Low_Voltage_BTN(%)',
    'District_Low_Voltage_BTE(%)'
]

# Calcular média por distrito
df_district_res = df.groupby('District')[residential_vars].mean().reset_index()
df_district_res['District'] = df_district_res['District'].str.title()

# Índice residencial ponderado
df_district_res['Residential_Index'] = (
    2 * df_district_res['District_Low_Voltage_BTN(%)'] +
    0.5 * df_district_res['District_Low_Voltage_BTE(%)']
)

# Normalização de 0 a 100
df_district_res['Residential_Index_norm'] = 100 * df_district_res['Residential_Index'] / df_district_res['Residential_Index'].max()

# Top 5 distritos mais residenciais
top5_res_high = df_district_res.sort_values(by='Residential_Index_norm', ascending=False).head(5)
print("Top 5 Most Residential Districts:")
print(top5_res_high[['District', 'Residential_Index_norm']])

# Top 5 distritos menos residenciais
top5_res_low = df_district_res.sort_values(by='Residential_Index_norm', ascending=True).head(5)
print("\nTop 5 Least Residential Districts:")
print(top5_res_low[['District', 'Residential_Index_norm']])


In [ ]:
sns.set(style='whitegrid')
top5_industrial = ['Beja', 'Portalegre', 'Aveiro', 'Leiria', 'Évora']
top5_residential = ['Setúbal', 'Viana Do Castelo', 'Vila Real', 'Faro', 'Coimbra']

def assign_upac(row):
    if row['Voltage level'] in ['BTN', 'BTE']:
        return 'Residential'
    else:
        return 'Industrial'

df['UPAC_Type'] = df.apply(assign_upac, axis=1)

df_top_industrial = df[df['District'].str.title().isin(top5_industrial)]
df_top_residential = df[df['District'].str.title().isin(top5_residential)]

agg_industrial = df_top_industrial.groupby(['Year','District','UPAC_Type'], as_index=False)['Total installed power (kW)'].sum()
agg_residential = df_top_residential.groupby(['Year','District','UPAC_Type'], as_index=False)['Total installed power (kW)'].sum()

def plot_stacked_bar(df_agg, title):
    fig, axes = plt.subplots(1, len(df_agg['District'].unique()), figsize=(20,6), sharey=True)

    if len(df_agg['District'].unique()) == 1:
        axes = [axes]

    for ax, district in zip(axes, df_agg['District'].unique()):
        data = df_agg[df_agg['District'] == district].pivot(index='Year', columns='UPAC_Type', values='Total installed power (kW)').fillna(0)
        data.plot(kind='bar', stacked=True, ax=ax, color=['#64b864','#f15e47'])
        ax.set_title(district)
        ax.set_ylabel('Total Installed Power (kW)')
        ax.set_xlabel('Year')
        ax.legend(title='UPAC Type')
        ax.grid(alpha=0.3)

    fig.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

plot_stacked_bar(agg_industrial, 'Top 5 Most Industrialized Districts')
plot_stacked_bar(agg_residential, 'Top 5 Most Residential Districts')


**Stacked Barplot of Technology Type Counts per Voltage Level**

In [ ]:
dominant_techs = ['Solar', 'Not Assigned']
df_dominant = df[df['Technology Type'].isin(dominant_techs)].copy()
df_others = df[~df['Technology Type'].isin(dominant_techs)].copy()

color_map = px.colors.qualitative.Vivid
color_dict = {tech: color_map[i % len(color_map)] for i, tech in enumerate(df['Technology Type'].unique())}

tech_counts = df.groupby(['Voltage level','Technology Type']).size().reset_index(name='Count')
dominant_df = tech_counts[tech_counts['Technology Type'].isin(dominant_techs)].copy()
others_df = tech_counts[~tech_counts['Technology Type'].isin(dominant_techs)].copy()

num_levels = df['Voltage level'].nunique()
fig_height = max(450, num_levels*60)

fig6 = make_subplots(
    rows=1, cols=2,
    shared_yaxes=True,
    column_widths=[0.6, 0.4],
    horizontal_spacing=0.05,
    subplot_titles=("Solar & Not Assigned", "Other Technologies")
)

for tech in dominant_techs:
    data = dominant_df[dominant_df['Technology Type'] == tech]
    fig6.add_trace(
        go.Bar(
            y=data['Voltage level'],
            x=data['Count'],
            name=tech,
            text=data['Count'],
            textposition='inside',
            orientation='h',
            marker=dict(color=color_dict[tech], line=dict(width=0.8, color='rgba(0,0,0,0.2)')),
            customdata=data['Technology Type'],
            hovertemplate="<b>Technology:</b> %{customdata}<br><b>Voltage Level:</b> %{y}<br><b>Installations:</b> %{x}<extra></extra>"
        ),
        row=1, col=1
    )

for tech in sorted(others_df['Technology Type'].unique()):
    data = others_df[others_df['Technology Type'] == tech]
    fig6.add_trace(
        go.Bar(
            y=data['Voltage level'],
            x=data['Count'],
            name=tech,
            text=data['Count'],
            textposition='inside',
            orientation='h',
            marker=dict(color=color_dict[tech], line=dict(width=0.8, color='rgba(0,0,0,0.2)')),
            customdata=data['Technology Type'],
            hovertemplate="<b>Technology:</b> %{customdata}<br><b>Voltage Level:</b> %{y}<br><b>Installations:</b> %{x}<extra></extra>"
        ),
        row=1, col=2
    )

fig6.update_layout(
    template='plotly_white',
    title="<b>Installations by Voltage Level and Technology Type</b>",
    barmode='stack',
    width=1300,
    height=fig_height,
    legend_title="<b>Technology Type</b>",
    margin=dict(l=120, r=60, t=90, b=60)
)
fig6.update_xaxes(title_text='Number of Installations', row=1, col=1)
fig6.update_xaxes(title_text='Number of Installations', row=1, col=2)
fig6.update_yaxes(title_text='Voltage Level', row=1, col=1)
fig6.update_yaxes(showticklabels=False, row=1, col=2)

fig6.show()


**Total Installed Power by Technology Type Over Time**

In [ ]:
dominant_techs = ['Solar', 'Not Assigned']
df_time = df.groupby(['Quarter', 'Technology Type'], as_index=False)['Total installed power (kW)'].sum()
df_dom = df_time[df_time['Technology Type'].isin(dominant_techs)]
df_others = df_time[~df_time['Technology Type'].isin(dominant_techs)]


fig8 = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Solar & Not Assigned', 'Other Technologies'],
    shared_yaxes=False
)

for tech in dominant_techs:
    data = df_dom[df_dom['Technology Type'] == tech]
    fig8.add_trace(
        go.Scatter(
            x=data['Quarter'],
            y=data['Total installed power (kW)'],
            mode='lines+markers',
            name=tech
        ),
        row=1, col=1
    )

for tech in df_others['Technology Type'].unique():
    data = df_others[df_others['Technology Type'] == tech]
    fig8.add_trace(
        go.Scatter(
            x=data['Quarter'],
            y=data['Total installed power (kW)'],
            mode='lines+markers',
            name=tech
        ),
        row=1, col=2
    )

fig8.update_layout(
    template='plotly_white',
    title='Total Installed Power by Technology Type Over Time',
    title_x=0.5,
    height=600,
    width=1200
)
fig8.show()